In [ ]:
import os
import json
import sqlite3
import numpy as np
import pandas as pd

from bertopic import BERTopic

from utils.analysis import load_and_prep_data

import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
tqdm.pandas()

### Load & prep data:
- `text_embedding_path`: path to new-line JSON file of document emebddings.
- `data_path`: path to processed CSV file containing analytic sample.
- `sql_path`: path to SQL database to grab additional columns.
- `topic_mapper_path`: path to JSON file mapping observation IDs to topic groups.
- `score_mapper_path`: path to JSON file mapping observation IDs to updated scores.

In [ ]:
data_path = os.path.join(
    "..",
    "data",
    "topic_data",
    "topic_data.csv"
)

sql_path = os.path.join(
    "..", 
    "data", 
    "sqlite", 
    "idw_reddit.db"
)

topic_mapper_path = os.path.join(
    "..", 
    "data", 
    "topic_data", 
    "labels", 
    "topic_mapper.json"
)

score_mapper_path = os.path.join(
    "..", 
    "data", 
    "updated_scores", 
    "score_mapper.json"
)

# load & prep data:
df = load_and_prep_data(
    data=data_path, 
    sql_db=sql_path, 
    topic_group_file=topic_mapper_path, 
    score_file=score_mapper_path
)

**Load BERtopic model:**

In [ ]:
NEW_MODEL = "updated_bertopic_model"
NEW_MODEL_PATH = os.path.join("..", "data", "topic_data", NEW_MODEL)
topic_model = BERTopic.load(NEW_MODEL_PATH)

In [ ]:
topic_model.get_topic_info()

**Topic representations (for mapping words to topic IDs):**

In [ ]:
topic_reps = dict(
    zip(
        topic_model.get_topic_info()["Topic"],
        topic_model.get_topic_info()["Representation"]
    )
)

topic_reps = {k:v for k,v in topic_reps.items() if k != -1}

In [ ]:
covid_topics = [1, 6, 29, 63, 74, 96, 115, 167, 168, 178, 194, 200, 215, 224, 234, 236, 242]
{k:",".join(v[:7]) for k,v in topic_reps.items() if k in covid_topics}

### Annotation Extracts

**Covid-19 relevant topics:**

`[1, 6, 29, 63, 74, 96, 115, 167, 168, 178, 194, 200, 215, 224, 236, 242]`

In [ ]:
covid_topics = [1, 6, 29, 63, 74, 96, 115, 167, 168, 178, 194, 200, 215, 224, 234, 236, 242]
covid_df = df.loc[df["new_topic"].isin(covid_topics)]
covid_df["date"] = pd.to_datetime(covid_df["date"])
covid_df.tail()

In [ ]:
covid_counts = topic_model.get_topic_info()
covid_counts = covid_counts[covid_counts["Topic"].isin(covid_topics)]
covid_counts

In [ ]:
timeline = covid_df.groupby("month_year").size().reset_index(name="count")

(
    alt.Chart(timeline).mark_line().encode(
        x=alt.X("month_year:N", title="Month-Year"),
        y=alt.Y("count:Q", title="Count"),
        tooltip=["month_year", "count"]
    )
)


In [ ]:
for topic in covid_topics:
    highest, lowest = (
        covid_df.loc[covid_df["new_topic"]==topic]["score"].max(), 
        covid_df.loc[covid_df["new_topic"]==topic]["score"].min()
    )
    
    n_above = len(
        covid_df.loc[
            (covid_df["new_topic"]==topic) &
            (covid_df["score"] > 1)
        ]
    )
    
    n_below = len(
        covid_df.loc[
            (covid_df["new_topic"]==topic) &
            (covid_df["score"] < 1)
        ]
    )
    
    print(f"Topic {topic} :: >1 = {n_above} ({highest}), <1 = {n_below} ({lowest})")

In [ ]:
# filter to January 2020 and forwards:
start_obs = pd.to_datetime("2020-01-01")
df["date"] = pd.to_datetime(df["date"])
covid_df = df.loc[df["new_topic"].isin(covid_topics)].copy()
covid_df = covid_df.loc[covid_df["date"] >= start_obs]

# only keep documents that are similar to topic vector:
covid_df = covid_df.loc[covid_df["topic_sim"] >= 0.5]

In [ ]:
covid_df

In [ ]:
# top 400 highest & lowest:
N = 400
highest = covid_df.sort_values("score", ascending=False).head(N)
highest["sample_source"] = "highest"

lowest = covid_df.sort_values("score", ascending=True).head(N)
lowest["sample_source"] = "lowest"

# random sample 200 that are NOT in the existing set:
random_n = 200
random_state = 42
hi_low_ids = highest["full_id"].tolist() + lowest["full_id"].tolist()
random_smpl = covid_df.loc[~covid_df["full_id"].isin(hi_low_ids)].sample(n=random_n, random_state=random_state)
random_smpl["sample_source"] = "random"

# concatenate:
sample_df = pd.concat([highest, lowest, random_smpl])
assert len(sample_df["full_id"].unique() == len(sample_df)), "Duplicates found!"
assert (N*2 + random_n) == len(sample_df["full_id"].unique()), "Duplicates found across samples!"
assert sample_df["topic_sim"].min() >= 0.50, "Similarity error!"

In [ ]:
sample_df

**Get raw texts:**

In [ ]:
db = os.path.join(
    "..",
    "data",
    "sqlite",
    "idw_reddit.db"
)

sql_df = pd.DataFrame()

conn = sqlite3.connect(db)
tables = ["comments", "posts"]

for table in tables:
    if table == "posts":
        temp = pd.read_sql(f"SELECT full_id, title, selftext FROM {table}", conn)
        temp.fillna(" ", inplace=True)
        temp["text"] = temp["title"] + " " + temp["selftext"]
        temp = temp[["full_id", "text"]]
    else:
        temp = pd.read_sql(f"SELECT full_id, body FROM {table}", conn)
        temp.rename(columns={"body": "text"}, inplace=True)
    sql_df = pd.concat([sql_df, temp])

conn.close()

In [ ]:
sql_df.head(10)

In [ ]:
sample_df = sample_df.merge(sql_df, on="full_id", how="left")
sample_df["url"] = "https://reddit.com" + sample_df["permalink"]
sample_df = sample_df[["url", "sample_source", "score", "full_id", "unique_id", "new_topic", "topic_sim", "score", "date", "month_year", "text"]]

In [ ]:
sample_df

In [ ]:
timeline = sample_df.groupby("month_year").size().reset_index(name="count")

(
    alt.Chart(timeline).mark_line().encode(
        x=alt.X("month_year:N", title="Month-Year"),
        y=alt.Y("count:Q", title="Count"),
        tooltip=["month_year", "count"]
    )
)


In [ ]:
sample_df["date"] = sample_df["date"].dt.strftime("%Y-%m-%d")
output = sample_df.to_dict("records")

with open("../doccano/datasets/idw_subreddit_covid_topics.jsonl", "w") as f:
    for line in tqdm(output):
        f.write(json.dumps(line) +"\n")